### Chatbots with LlamaIndex  
LlamaIndex serves as a bridge between your data and Large Language Models (LLMs), providing a toolkit that enables you to establish a query interface around your data for a variety of tasks, such as question-answering and summarization.

In this tutorial, we'll walk you through building a context-augmented chatbot  

#### Installing Packages

In [6]:
!pip install -q openai
!pip install -q llama-index
!pip install -q llama-index-experimental
!pip install -q pypdf
!pip install -q docx2txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-experimental 0.1.3 requires llama-index-core<0.11.0,>=0.10.11.post1, but you have llama-index-core 0.12.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-parse 0.5.14 requires llama-index-core>=0.11.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index 0.12.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-retrievers-bm25 0.5.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10.68.post1 which is incompatible.
llama-index-readers-llama-parse 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.10

#### Importing Packages

In [7]:
import os
import openai

#os.environ["OPENAI_API_KEY"] = "<the key>"
openai.api_key = os.environ["OPENAI_API_KEY"]

import sys
import shutil
import glob
import logging
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

import pandas as pd

import llama_index

## Llamaindex readers
from llama_index.core import SimpleDirectoryReader

## LlamaIndex Index Types
from llama_index.core import ListIndex
from llama_index.core import VectorStoreIndex
from llama_index.core import TreeIndex
from llama_index.core import KeywordTableIndex
from llama_index.core import SimpleKeywordTableIndex
from llama_index.core import DocumentSummaryIndex
from llama_index.core import KnowledgeGraphIndex
from llama_index.experimental.query_engine import PandasQueryEngine


## LlamaIndex Context Managers
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.response_synthesizers import ResponseMode
from llama_index.core.schema import Node

## LlamaIndex Templates
from llama_index.core.prompts import PromptTemplate
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.base.llms.types import ChatMessage, MessageRole

## LlamaIndex Callbacks
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler

In [8]:
print("LLamaIndex:", llama_index.core.__version__)
print("OpenAI:", openai.__version__)

LLamaIndex: 0.10.68.post1
OpenAI: 1.55.3


In [9]:
import logging

#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
#logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

#### Defining Models

In [10]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

#model="gpt-4o"
model="gpt-4o-mini"

Settings.llm = OpenAI(temperature=0, 
                      model=model, 
                      #max_tokens=512
                      PRESENCE_PENALTY=-2,
                      TOP_P=1,
                     )

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")
#Settings.llm = Ollama(model="llama3.2", request_timeout=300.0)
#Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

### Using the the vector index created in the notebook: `RAG_LLamaIndex.ipynb`

#### Defining Folders

In [15]:
PERSIST_DIR = "../../Index/VectorStoreIndex/"

#### Retrieving Vector Store Index  

In [16]:
vectorstoreindex = load_index_from_storage(
    storage_context=StorageContext.from_defaults(persist_dir=PERSIST_DIR)
)

## Retrieving your data  

#### Retrieving from [Vector Store Index](https://docs.llamaindex.ai/en/stable/api_reference/query/retrievers/vector_store.html)  


In [17]:
query_engine = vectorstoreindex.as_query_engine(similarity_top_k=3,
                                                retriever_mode="embedding",
                                                #response_mode="accumulate",
                                                response_mode="compact",
                                                #response_mode="tree_summarize",
                                                verbose=True)
response = query_engine.query("Who is Gregor?")
print(response)

Gregor is a character who has undergone a transformation, leading to a significant change in his life and circumstances. He is depicted as being confined to his room, where he struggles with his new condition and the impact it has on his family. Gregor is also shown to have a close relationship with his sister, Grete, who takes on the responsibility of caring for him, despite the challenges they face. His previous role as the family's provider is contrasted with his current state, where he feels isolated and disconnected from his family.


In [18]:
query_engine = vectorstoreindex.as_query_engine(similarity_top_k=3,
                                                retriever_mode="embedding",
                                                #response_mode="accumulate",
                                                #response_mode="compact",
                                                response_mode="tree_summarize",
                                                verbose=True)

response = query_engine.query("Who is Paul McCartney?")
print(response)

The provided information does not contain any details about Paul McCartney.


In [20]:
query_engine = vectorstoreindex.as_query_engine(similarity_top_k=3,
                                                retriever_mode="embedding",
                                                response_mode="accumulate",
                                                #response_mode="compact",
                                                #response_mode="tree_summarize",
                                                verbose=True)

response = query_engine.query("how many stars in the galaxy of auron-b?")
print(response)

Response 1: The provided information does not contain any details about the galaxy of Auron-B or the number of stars within it. Therefore, I cannot provide an answer to that query.
---------------------
Response 2: The provided context does not contain any information regarding the number of stars in the galaxy of Auron-B.
---------------------
Response 3: The provided context does not contain any information regarding the number of stars in the galaxy of Auron-B.


In [21]:
query_engine = vectorstoreindex.as_query_engine(similarity_top_k=3,
                                                retriever_mode="embedding",
                                                #response_mode="accumulate",
                                                response_mode="compact",
                                                #response_mode="tree_summarize",
                                                verbose=True)

response = query_engine.query("how many stars in the galaxy of auron-b?")
print(response)

The provided context does not contain any information regarding the number of stars in the galaxy of Auron-B.


## Creating an Simple Interactive Chatbot for our Index  

Chat Modes have their own different [retrieval modes](https://www.actalyst.ai/blog/optimizing-enterprise-chatbots-with-llamaindex-chat-modes-in-a-rag-system)

![](https://assets-global.website-files.com/652165b2d773cd686ae910a6/652e2b0154e3af353e2d9422_actalyst-llalaindex-chatmodes.png)

In [23]:
chat_engine = vectorstoreindex.as_chat_engine(chat_mode="context", 
                                              verbose=False)
chat_engine.reset()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  What is the book about?


Assistant: "The Metamorphosis" by Franz Kafka is a novella that tells the story of Gregor Samsa, a traveling salesman who wakes up one morning to find himself transformed into a giant insect. The narrative explores themes of alienation, identity, and the absurdity of life as Gregor struggles to adapt to his new condition while dealing with the reactions of his family and society. As he becomes increasingly isolated and unable to fulfill his role as the breadwinner, the story delves into the impact of his transformation on his relationships and the existential questions surrounding his existence. The novella is often interpreted as a commentary on the human condition, the nature of work, and the struggle for acceptance.



Human:  Who is Ringo Star?


Assistant: Ringo Starr, born Richard Starkey on July 7, 1940, is an English musician, songwriter, and actor best known as the drummer for the iconic rock band The Beatles. He joined the band in 1962 and contributed to their distinctive sound with his unique drumming style and occasional lead vocals on songs like "With a Little Help from My Friends" and "Yellow Submarine." 

After The Beatles disbanded in 1970, Ringo Starr pursued a successful solo career, releasing several albums and hit singles. He is also known for his work in film and television, including appearances in movies and as a narrator for children's programs. Ringo Starr is celebrated not only for his musical contributions but also for his charismatic personality and enduring influence in the music industry. He was inducted into the Rock and Roll Hall of Fame as a member of The Beatles in 1988 and received a second induction as a solo artist in 2015.



Human:  how many stars in the galaxy of auron-b?


Assistant: The galaxy of Auron-B does not exist in known astronomical terms. It may be a fictional galaxy from a book, movie, or other media. If you are referring to a specific work of fiction or a particular context, please provide more details, and I would be happy to help!



Human:  bye


Assistant: Goodbye! If you have more questions in the future, feel free to ask. Have a great day!



Human:  exit


In [24]:
chat_engine = vectorstoreindex.as_chat_engine(chat_mode="condense_question", verbose=False)
chat_engine.reset()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  What is the book about?


Assistant: The book revolves around Gregor Samsa, who wakes up one morning to find himself transformed into a monstrous insect. The narrative explores his struggles with this drastic change, his attempts to communicate with his family, and their reactions to his condition. As Gregor grapples with his new physical form and the challenges it presents, themes of alienation, identity, and the burdens of responsibility emerge, highlighting the impact of his transformation on both himself and his family.



Human:  Who is Ringo Star?


Assistant: The query pertains to Ringo Starr, a member of the Beatles, and his connection to themes of alienation and identity in literature. However, the provided context does not contain any information about Ringo Starr or his relationship to these themes. Therefore, I cannot provide an answer based on the given context.



Human:  how many stars in the galaxy of auron-b?


Assistant: The information provided does not contain any details about the galaxy of Auron-B or the number of stars it may have. Therefore, I cannot provide an answer to that query.



Human:  exit


---
### We can see that, aparently, there is no consistence on the usage of the context and external knowledge. We can check it further on these pages:  

[LlamaIndex Prompts](https://docs.llamaindex.ai/en/stable/module_guides/models/prompts/), [
LlamaIndex Chat Prompts Customization](https://docs.llamaindex.ai/en/stable/examples/customization/prompts/chat_prompts/), and [default_prompts.py](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/prompts/default_prompts.py)  

### Creating an [Customized Prompt](https://github.com/run-llama/llama_index/blob/main/llama-index-core/llama_index/core/prompts/chat_prompts.py) Chatbot

In [26]:
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType

In [32]:
CUSTOM_PROMPT = [
    ChatMessage(
        content=(
            "You are an expert Q&A system that is trusted around the world.\n"
            "Always answer the query using only the provided context information, "
            "Do not use prior knowledge.\n"
            "Some rules to follow:\n"
            "1. Never directly reference the given context in your answer.\n"
            "2. Avoid statements like 'Based on the context, ...' or "
            "'The context information ...' or anything along "
            "those lines."
        ),
        role=MessageRole.SYSTEM,
    ),
    ChatMessage(
        content=(
            "Context information from multiple sources is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the information from multiple sources and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
        role=MessageRole.USER,
    ),
]


CHAT_PROMPT = ChatPromptTemplate(message_templates=CUSTOM_PROMPT)

In [33]:
chat_engine = vectorstoreindex.as_chat_engine(chat_mode="condense_question",
                                              verbose=True,
                                              text_qa_template=CHAT_PROMPT)
chat_engine.reset()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Hi, what is your name?


Querying with: Hi, what is your name?
Assistant: I don't have a name.



Human:  What is the book about?


Querying with: What is the book about that you mentioned earlier?
Assistant: The book revolves around Gregor Samsa, who wakes up one morning to find himself transformed into a large insect. The narrative explores his struggles with this new condition, his relationships with his family, and the impact of his transformation on their lives. As Gregor attempts to navigate his altered existence, themes of isolation, identity, and the burden of familial responsibilities are examined. The story delves into the emotional and psychological turmoil experienced by Gregor and his family as they confront the changes in their dynamics and the challenges posed by his new form.



Human:  Who is Paul MacCartney?


Querying with: Who is Paul McCartney and what is his significance in music?
Assistant: The provided context does not contain any information about Paul McCartney or his significance in music. Therefore, I cannot provide an answer to that query.



Human:  How the insect looks like?


Querying with: What does the insect that Gregor Samsa transforms into look like?
Assistant: The description of the insect that Gregor Samsa transforms into is not provided in the information given. Therefore, there are no details available regarding its appearance.



Human:  exit


### Adding Memory and Custom System Prompt

In [35]:
from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=80000)
chat_engine = vectorstoreindex.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
            "Context information from multiple sources is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the information from multiple sources"
            "answer the query.\n"
            "If the query is unrelated to the context, just answer: I don't know"
            "Always start your answer with 'Dear Student'" 
            "Query: {query_str}\n"
            "Answer: "
    ),
)

chat_engine.reset()
chat_engine.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.



Human:  Hi, what is your name?


Assistant: Dear Student, I don't know.



Human:  Who is Gregor?


Assistant: Dear Student, Gregor Samsa is the main character in Franz Kafka's novella "Metamorphosis." He is a traveling salesman who wakes up one morning to find himself transformed into a giant insect. The story explores themes of alienation, identity, and the impact of this transformation on Gregor's family and his own sense of self.



...

Human:  Can you use information outside the context?


Assistant: Dear Student, I don't know.



Human:  How many stars are in the universe?


Assistant: Dear Student, I don't know.



Human:  exit


In [36]:
memory.to_dict()

{'chat_store': {'store': {'chat_history': [{'role': <MessageRole.USER: 'user'>,
     'content': 'Hi, what is your name?',
     'additional_kwargs': {}},
    {'role': <MessageRole.ASSISTANT: 'assistant'>,
     'content': "Dear Student, I don't know.",
     'additional_kwargs': {}},
    {'role': <MessageRole.USER: 'user'>,
     'content': 'Who is Gregor?',
     'additional_kwargs': {}},
    {'role': <MessageRole.ASSISTANT: 'assistant'>,
     'content': 'Dear Student, Gregor Samsa is the main character in Franz Kafka\'s novella "Metamorphosis." He is a traveling salesman who wakes up one morning to find himself transformed into a giant insect. The story explores themes of alienation, identity, and the impact of this transformation on Gregor\'s family and his own sense of self.',
     'additional_kwargs': {}},
    {'role': <MessageRole.USER: 'user'>,
     'content': 'How is the appearance of the insect?',
     'additional_kwargs': {}},
    {'role': <MessageRole.ASSISTANT: 'assistant'>,
   